In [3]:
import requests
import json
import torch
import os
from tqdm import tqdm

# Load SQuAD Data

In [4]:
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

File ‘train-v2.0.json’ already there; not retrieving.

File ‘dev-v2.0.json’ already there; not retrieving.



In [5]:
# Load the training dataset and take a look at it
with open('train-v2.0.json', 'rb') as f:
  squad = json.load(f)

In [6]:
def read_data(path):
  # load the json file
  with open(path, 'rb') as f:
    squad = json.load(f)

  contexts = []
  questions = []
  answers = []

  for group in squad['data']:
    for passage in group['paragraphs']:
      context = passage['context']
      for qa in passage['qas']:
        question = qa['question']
        for answer in qa['answers']:
          contexts.append(context)
          questions.append(question)
          answers.append(answer)

  return contexts, questions, answers

In [7]:
train_contexts, train_questions, train_answers = read_data('train-v2.0.json')
valid_contexts, valid_questions, valid_answers = read_data('dev-v2.0.json')

In [9]:
train_contexts[0]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [12]:
train_answers[0]

{'text': 'in the late 1990s', 'answer_start': 269}

In [13]:
train_questions[0]

'When did Beyonce start becoming popular?'

In [19]:
import openai
client = openai.OpenAI(
    base_url="http://localhost:11434/v1", # ollama endpoint
    api_key='NO_KEY'
    )

# Few shot inference using llama3.2:1b

In [22]:
def get_predicted_answer(cntxt,qstn):
    completion = client.chat.completions.create(

        model= 'llama3.2:1b',
        messages=[
            {"role": "system", "content": """
            You are an AI assistant in Question Answering. Your task is to generate Answer based on a given context.
            
            Here are some examples:
            
            Context: The Great Wall of China is a series of fortifications that were built across the historical northern borders of ancient Chinese states and Imperial China. The wall was built to protect against invasions and raids. 
            Question: What was the purpose of the Great Wall of China?
            Answer: To protect against invasions and raids.
            
            Context: The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. 
            Question: Who designed the Eiffel Tower?
            Answer: Gustave Eiffel.
            
            Context: The Mona Lisa is a portrait painting by Leonardo da Vinci, considered an archetypal masterpiece of the Italian Renaissance. It is housed in the Louvre Museum in Paris.
            Question: Who painted the Mona Lisa?
            Answer: Leonardo da Vinci.
            
            # Now, the actual context and question for inference, respond with only the Answer and nothing else
            """},
            {"role": "user", "content": f"""
            Context: {cntxt}
            Question: {qstn}
            """}
        ]
    )
    return completion.choices[0].message.content

In [25]:
i = 10
question = train_questions[i]
context = train_contexts[i]
answer = train_answers[i]
ans = get_predicted_answer(context,question)

print('*'*100)
print('Question',context)
print('*'*100)
print('Original Answer:',answer)
print('*'*100)
print('Model Answer:',ans)

****************************************************************************************************
Question Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
****************************************************************************************************
Original Answer: {'text': 'Dangerously in Love', 'answer_start': 505}
************************

In [26]:
i = 21
question = train_questions[i]
context = train_contexts[i]
answer = train_answers[i]
ans = get_predicted_answer(context,question)

print('*'*100)
print('Question',context)
print('*'*100)
print('Original Answer:',answer)
print('*'*100)
print('Model Answer:',ans)

****************************************************************************************************
Question Following the disbandment of Destiny's Child in June 2005, she released her second solo album, B'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio al

In [27]:
i = 25
question = train_questions[i]
context = train_contexts[i]
answer = train_answers[i]
ans = get_predicted_answer(context,question)

print('*'*100)
print('Question',context)
print('*'*100)
print('Original Answer:',answer)
print('*'*100)
print('Model Answer:',ans)

****************************************************************************************************
Question Following the disbandment of Destiny's Child in June 2005, she released her second solo album, B'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio al